In [1]:
// To see less warnings
import org.apache.log4j._
Logger.getLogger("org").setLevel(Level.ERROR)

Intitializing Scala interpreter ...

Spark Web UI available at http://74e4be6e0134:4042
SparkContext available as 'sc' (version = 3.2.1, master = local[*], app id = local-1651734057756)
SparkSession available as 'spark'


import org.apache.log4j._


In [2]:
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}

import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit}


In [3]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.regression.LinearRegression

val spark = SparkSession.builder()
                        .appName("LinearRegEval1")
                        .config("spark.master", "local")
                        .getOrCreate()

import spark.implicits._

import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.regression.LinearRegression
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@4273a12b
import spark.implicits._


In [4]:
// Setting Up DataFrame for Machine Learning


import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors


// Prepare training and test data.
val data = spark.read.option("header","true").option("inferSchema","true").format("csv").load("USA_Housing.csv")


// Grab only numerical columns from the data
var df = data.select($"Price", $"Avg Area Income", $"Avg Area House Age", $"Avg Area Number of Rooms", $"Area Population")
// Convert to double 
df = df.withColumn("Avg Area Income", df("Avg Area Income").cast("double"))
df = df.withColumn("Avg Area House Age", df("Avg Area Income").cast("double"))
df = df.withColumn("Avg Area Number of Rooms", df("Avg Area Number of Rooms").cast("double"))
df = df.withColumn("Area Population", df("Area Population").cast("double"))

df = df.withColumn("Price", df("Price").cast("double"))

df = df.withColumnRenamed("Price", "label")



import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
data: org.apache.spark.sql.DataFrame = [Avg Area Income: double, Avg Area House Age: double ... 4 more fields]
df: org.apache.spark.sql.DataFrame = [label: double, Avg Area Income: double ... 3 more fields]
df: org.apache.spark.sql.DataFrame = [label: double, Avg Area Income: double ... 3 more fields]
df: org.apache.spark.sql.DataFrame = [label: double, Avg Area Income: double ... 3 more fields]
df: org.apache.spark.sql.DataFrame = [label: double, Avg Area Income: double ... 3 more fields]
df: org.apache.spark.sql.DataFrame = [label: double, Avg Area Income: double ... 3 more fields]
df: org.apache.spark.sql.DataFrame = [label: double, Avg Area Income: double ... 3 more fields]
df: org.apache.sp...


In [5]:
// An assembler converts the input values to a vector
// A vector is what the ML algorithm reads to train a model

// Set the input columns from which we are supposed to read the values
// Set the name of the column where the vector will be stored
val assembler = new VectorAssembler().setInputCols(Array("Avg Area Income","Avg Area House Age","Avg Area Number of Rooms","Area Population")).setOutputCol("features")

// Use the assembler to transform our DataFrame to the two columns
val df1 = assembler.transform(df).select($"label", $"features")

assembler: org.apache.spark.ml.feature.VectorAssembler = VectorAssembler: uid=vecAssembler_7dfb021af0b2, handleInvalid=error, numInputCols=4
df1: org.apache.spark.sql.DataFrame = [label: double, features: vector]


In [6]:
val Array(training, test) = df1.randomSplit(Array(0.7, 0.3), seed = 12345)

training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: double, features: vector]


In [8]:
// LINEAR REGRESSION 
val lr = new LinearRegression()


// PARAMETER GRID BUILDER
val paramGrid = new ParamGridBuilder().addGrid(lr.regParam,Array(1000,0.001)).build()

// TRAIN TEST SPLIT

// In this case the estimator is simply the linear regression.
// A TrainValidationSplit requires an Estimator, a set of Estimator ParamMaps, and an Evaluator.
// 80% of the data will be used for training and the remaining 20% for validation.
val trainValidationSplit = (new TrainValidationSplit()
                            .setEstimator(lr)
                            .setEvaluator(new RegressionEvaluator())
                            .setEstimatorParamMaps(paramGrid)
                            .setTrainRatio(0.8) )


// You can then treat this object as the new model and use fit on it.
// Run train validation split, and choose the best set of parameters.
val model = trainValidationSplit.fit(training)


// EVALUATION USING THE TEST DATA

// Make predictions on test data. model is the model with combination of parameters
// that performed best.
model.transform(test).select("features", "label", "prediction").show()

// Check out the metrics
model.validationMetrics


+--------------------+-----------+------------------+
|            features|      label|        prediction|
+--------------------+-----------+------------------+
|[48904.98327,4890...|201898.0866| 563666.4093469658|
|[49851.13478,4985...|283208.1322| 554043.4204602668|
|[47685.25759,4768...|294170.7464|418509.62306271656|
|[59801.49103,5980...|311111.2006| 748840.5853979208|
|[59141.79644,5914...|313651.5032| 577790.0929296722|
|[43952.33621,4395...| 324981.993| 569285.1399972925|
|[58342.36779,5834...|365929.5973| 567490.5357962397|
|[50847.11311,5084...|398909.5099| 621924.4535205176|
|[52202.85372,5220...|404976.3659| 647514.5952812966|
|[42814.99304,4281...|452530.1767| 637844.4839183541|
|[59788.21893,5978...|461473.5665| 620147.7395165844|
|[62639.15981,6263...|476971.4559| 785938.1802047442|
|[66989.50469,6698...|490338.6033|  684365.693746607|
|[55415.24226,5541...|492984.5947|472768.54618370044|
|[41007.45867,4100...|494742.5436| 563661.6286209642|
|[44806.00516,4480...|497579

lr: org.apache.spark.ml.regression.LinearRegression = linReg_ef4ee4f9aea2
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	linReg_ef4ee4f9aea2-regParam: 1000.0
}, {
	linReg_ef4ee4f9aea2-regParam: 0.001
})
trainValidationSplit: org.apache.spark.ml.tuning.TrainValidationSplit = tvs_d2dbcc67f195
model: org.apache.spark.ml.tuning.TrainValidationSplitModel = TrainValidationSplitModel: uid=tvs_d2dbcc67f195, bestModel=LinearRegressionModel: uid=linReg_ef4ee4f9aea2, numFeatures=4, trainRatio=0.8
res2: Array[Double] = Array(194769.4647996773, 194765.98554157064)
